In [55]:
import numpy as np
import pandas as pd

In [56]:
import pulp

In [57]:
def create_vars(_crops_year, df):

    return dict(zip(_crops_year.keys(),
            [pulp.LpVariable(i, 
                                lowBound = df.loc[_crops_year[i], 'lb'] if not np.isnan(df.loc[_crops_year[i], 'lb']) else None, 
                                upBound  = df.loc[_crops_year[i], 'ub'] if not np.isnan(df.loc[_crops_year[i], 'ub']) else None) 
                    for i in _crops_year.keys()]))

In [58]:
integer_variables = pd.read_csv('dados/al_rasheed/integer_variables.csv', sep=';')
crops = integer_variables.variable.to_list()
integer_variables.set_index(keys='variable', inplace=True)

In [59]:
income = pd.read_csv('dados/al_rasheed/net_income.csv', sep=';')
constraints = pd.read_csv('dados/al_rasheed/constraints.csv', sep=';')

In [60]:
YEARS = 1

In [61]:
crops_year = dict(zip([f"{i}y_{j}" for i in range(1,YEARS+1) for j in crops], YEARS*crops))
